<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Overview of Isentropic Analysis</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

Analysis and visualization in isentropic coordinates can be an important tool for synoptic meteorology. This notebook provides an introduction to isentropic coordinates and demonstrates basic plotting for isentropic analysis. 
<div style="float:right; width:250 px"><img src="../../instructors/images/isentropic_preview.png" alt="Example plot in isentropic coordinates" style="height: 400px;"></div>


### Focuses
* Understand the difference between isentropic and isobaric coordinates
* Demonstrate isentropic coordinate for synoptic meteorology
* Convert model output in isobaric coordinates to isentropic coordinates with [MetPy](https://unidata.github.io/MetPy/latest/index.html)
* Demonstrate visualizing this with [Matplotlib](https://matplotlib.org/)


### Objectives
1. [Understanding Isentropic Coordinates](#1.-Understanding-Isentropic-Coordinates)
1. [Interpolation to Isentropic Coordinates](#2.-Interpolation-to-Isentropic-Coordinates)
1. [Plotting Isentropic Coordinates](#3.-Plotting-Isentropic-Coordinates)
1. [See also](#4.-See-also)

---

### Imports
We will begin by importing the important packages to be used throughout this example. This example will use sample data included within the MetPy package. Further examples of isentropic interpolation and analysis can be found in the [MetPy documentation](https://unidata.github.io/MetPy/latest/examples/isentropic_example.html#sphx-glr-examples-isentropic-example-py) and in the [Unidata Python Gallery](https://unidata.github.io/python-gallery/examples/Isentropic_Interpolation.html#sphx-glr-examples-isentropic-interpolation-py)

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import add_timestamp
from metpy.units import units

---

## 1. Understanding Isentropic Coordinates

Gridded meteorological data, such as model output, is provided on a grid, regularly spaced horizontally, vertically, and in time. Horizontally, such data is typically projected to a geographic reference grid, such as the Lambert Conformal Conic, with common units of distance (typically meters). The vertical coordinate can also be geo-referenced in meters (either above mean sea level, MSL, or above ground level, AGL). However, it is more common to use units of pressure as the vertical coordinate. This is called *isobaric coordinates*, where height is referenced by decreasing pressure as altitude increases. This system is quite common in synoptic meteorology, and is often the system in which gridded data is provided.

Another choice of vertical reference is to use potential temperature rather than pressure. This is referred to as *isentropic coordinates*. Potential temperature increases with increasing height for a neutral or statically stable atmosphere. This means that a constant surface of potential temperature (called an *isentropic surface*) can be used as the vertical reference.

Just like isobaric surfaces are used for synoptic analysis (such as the 500-mb height analysis), an isentropic surface can also be used. A parcel of air will remain at the same potential temperature at it moves through the atmosphere since this movement is adiabatic. Therefore, air parcels will travel horizontally along isentropic surfaces. The advantage of isentropic analysis then is that if wind vectors are plotting, movement along the surface can be implied directly, rather than through geostrophic asumptions inherent in isobaric coordinates. Additionally, if pressure is contoured on the surface, vertical motion can be implied directly without the need for Quasi-Geostrophic assumptions.

For a more complete description of isentropic coordinates and its associated advantages and disadvantages, see this [presentation](https://www.atmos.illinois.edu/~snesbitt/ATMS505/stuff/07_Isentropic%20Analysis.pdf) from the University of Illinois.

<a href="#top">Top</a>

---

## 2. Interpolation to Isentropic Coordinates

To perform an anlysis on isentropic surfaces, we must first convert our data to isentropic coordinates. This is easily acomplished with `mpcalc.isentropic_interpolation`. Let's start by opening some sample data.

In [ ]:
data = xr.open_dataset(get_test_data('narr_example.nc', False))
data = data.squeeze().set_coords(['lon', 'lat'])

All that is required to interpolate to isentropic coordinates is to specify the surfaces we want.

In [ ]:
isentlevs = [296.] * units.kelvin
isent_data = mpcalc.isentropic_interpolation_as_dataset(
    isentlevs,
    data['Temperature'],
    data['u_wind'],
    data['v_wind'],
    data['Specific_humidity'],
    data['Geopotential_height']
)

To make plotting easier, we will perform some unit conversions and calculate relative humidity.

In [ ]:
# Convert wind to kt
isent_data['u_wind'] = isent_data['u_wind'].metpy.convert_units('kt')
isent_data['v_wind'] = isent_data['v_wind'].metpy.convert_units('kt')

# Calculate relative humidity
isent_data['Relative_humidity'] = mpcalc.relative_humidity_from_specific_humidity(
    isent_data['pressure'],
    isent_data['temperature'],
    isent_data['Specific_humidity']
).metpy.convert_units('percent')

That's it! Our original gridded model data in isobaric coordinates has been converted to isentropic coordinates. Now, on to plotting.

<a href="#top">Top</a>

---

## 3. Plotting Isentropic Coordinates


Plotting isentropic coordinates is much the same as for isobaric coordinates. A common analysis is to plot pressure, wind, and relative humidity on a constant isentropic surface. This can be accomplished through the use of `ax.contour`, `ax.contourf`, and `ax.barbs`.

In [ ]:
# Set up our projection and coordinates
crs = ccrs.LambertConformal(central_longitude=-100.0, central_latitude=45.0)
lon = isent_data['pressure'].metpy.longitude
lat = isent_data['pressure'].metpy.latitude

# Coordinates to limit map area
bounds = [(-122., -75., 25., 50.)]
# Choose a level to plot, in this case 296 K (our sole level in this example)
level = 0

fig = plt.figure(figsize=(17., 12.))
ax = fig.add_subplot(1, 1, 1, projection=crs)
ax.set_extent(*bounds, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.75)
ax.add_feature(cfeature.STATES, linewidth=0.5)

# Plot the surface
clevisent = np.arange(0, 1000, 25)
cs = ax.contour(lon, lat, isent_data['pressure'].isel(isentropic_level=level),
                clevisent, colors='k', linewidths=1.0, linestyles='solid',
                transform=ccrs.PlateCarree())
cs.clabel(fontsize=10, inline=1, inline_spacing=7, fmt='%i', rightside_up=True,
          use_clabeltext=True)

# Plot RH
cf = ax.contourf(lon, lat, isent_data['Relative_humidity'].isel(isentropic_level=level),
                 range(10, 106, 5), cmap=plt.cm.gist_earth_r, transform=ccrs.PlateCarree())
cb = fig.colorbar(cf, orientation='horizontal', aspect=65, shrink=0.5, pad=0.05,
                  extendrect='True')
cb.set_label('Relative Humidity', size='x-large')

# Plot wind barbs
ax.barbs(lon.values, lat.values, isent_data['u_wind'].isel(isentropic_level=level).values,
         isent_data['v_wind'].isel(isentropic_level=level).values, length=6,
         regrid_shape=20, transform=ccrs.PlateCarree())

# Make some titles
ax.set_title(f'{isentlevs[level]:~.0f} Isentropic Pressure (hPa), Wind (kt), '
             'Relative Humidity (percent)', loc='left')
add_timestamp(ax, isent_data['time'].values.astype('datetime64[ms]').astype('O'),
              y=0.02, high_contrast=True)
fig.tight_layout()

In the above image, it can be inferred that there is rising motion over much of New England where the wind vectors show motion from higher pressure toward lower pressure across pressure contours.

<a href="#top">Top</a>

---

## 4. See also
For a more complete description of isentropic coordinates and its associated advantages and disadvantages, see this [presentation](https://www.atmos.illinois.edu/~snesbitt/ATMS505/stuff/07_Isentropic%20Analysis.pdf) from the University of Illinois. Further examples of isentropic interpolation and analysis can be found in the [MetPy documentation](https://unidata.github.io/MetPy/latest/examples/isentropic_example.html#sphx-glr-examples-isentropic-example-py) and in the [Unidata Python Gallery](https://unidata.github.io/python-gallery/examples/Isentropic_Interpolation.html#sphx-glr-examples-isentropic-interpolation-py)

<a href="#top">Top</a>

---